In [2]:
import fitz
from tqdm import tqdm
        

In [3]:
path = "./textbooks"
pdf_file = path+"/textbook.pdf"
doc = fitz.open(pdf_file)

In [4]:
index_str = ""
for i in range(6, 16):
    index_str += doc[i].get_text()

In [10]:
from collections import defaultdict

index_ = index_str.split('\n')
chapters = defaultdict(dict)

for i in range(1,29):
    chap = f'CHAPTER {i}'
    sections = {}
    in_sections = {}
    for j in range(len(index_)):
        if index_[j] == chap:
            chapters[chap.replace(' ', '_')]['name'] = index_[j+1].replace(' ', '_')
            chapters[chap.replace(' ', '_')]['start_page'] = index_[j+2]
        
        if index_[j].startswith(f'{i}.'):
            sections[(index_[j].replace(' ', '_').replace('10', 'X').replace('11', 'Y'))] = index_[j+1]
            in_sections[index_[j]] = int(index_[j+1]) - int(chapters[chap.replace(' ', '_')]['start_page']) + 1

        if index_[j] == "Key Terms":
            chapters[chap.replace(' ', '_')]['end_page'] = int(index_[j+1])-1
                
        if index_[j] == f'CHAPTER {i+1}':
            chapters[chap.replace(' ', '_')]['sections'] = sections
            chapters[chap.replace(' ', '_')]['in_sections'] = in_sections
            break

chapters['CHAPTER_28']['sections'] = sections
        

In [50]:
doc[30].get_text().replace('\n', ' ').replace('Access for free at openstax.org', '')

'FIGURE 1.6 Metabolism Anabolic reactions are building reactions, and they consume energy. Catabolic reactions break materials down and release energy. Metabolism includes both anabolic and catabolic reactions. Every cell in your body makes use of a chemical compound, adenosine triphosphate (ATP), to store and release energy. The cell stores energy in the synthesis (anabolism) of ATP, then moves the ATP molecules to the location where energy is needed to fuel cellular activities. Then the ATP is broken down (catabolism) and a controlled amount of energy is released, which is used by the cell to perform a particular job. INTERACTIVE LINK View this animation (http://openstax.org/l/metabolic) to learn more about metabolic processes. Which organs of the body likely carry out anabolic processes? What about catabolic processes? Responsiveness Responsiveness is the ability of an organism to adjust to changes in its internal and external environments. An example of responsiveness to external s

In [11]:
chapters

defaultdict(dict,
            {'CHAPTER_1': {'name': 'An_Introduction_to_the_Human_Body',
              'start_page': '7',
              'end_page': 31,
              'sections': {'1.1_Overview_of_Anatomy_and_Physiology': '8',
               '1.2_Structural_Organization_of_the_Human_Body': '9',
               '1.3_Functions_of_Human_Life': '14',
               '1.4_Requirements_for_Human_Life': '16',
               '1.5_Homeostasis': '20',
               '1.6_Anatomical_Terminology': '23',
               '1.7_Medical_Imaging': '28'},
              'in_sections': {'1.1 Overview of Anatomy and Physiology': 2,
               '1.2 Structural Organization of the Human Body': 3,
               '1.3 Functions of Human Life': 8,
               '1.4 Requirements for Human Life': 10,
               '1.5 Homeostasis': 14,
               '1.6 Anatomical Terminology': 17,
               '1.7 Medical Imaging': 22}},
             'CHAPTER_2': {'end_page': 74,
              'name': 'The_Chemical_Level

In [7]:
import json

out_file = open("page_info.json", "w")
json.dump(chapters, out_file, indent=6)
out_file.close()

In [40]:
import os

doc = doc
index_end = 16

os.mkdir(f'./textbooks/textbook')

for chapter in chapters.keys():
    
    chapter_name = chapters[chapter]['name']
    sections = chapters[chapter]['sections']
    page = int(chapters[chapter]['start_page']) + index_end - 1
    sorted_index = sorted(sections)

    os.mkdir(f'./textbooks/textbook/{chapter}_{chapter_name}')

    text = ""
    print(sorted_index)
    if chapters[chapter]['start_page'] in sections.values():
        text += doc[page].get_text()
    else:
        while str((page - index_end + 1)) not in sections.values():
            text += doc[page].get_text()
            page += 1

    intro_file = open(f'./textbooks/textbook/{chapter}_{chapter_name}/introduction.txt', 'w', encoding='utf-8')
    intro_file.write(text)
    

    for i in range(len(sorted_index)):
        text = ""
        if i == len(sorted_index) - 1:
            end_page = int(chapters[chapter]['end_page']) + index_end - 1
        else:
            end_page = int(sections[sorted_index[i+1]]) + index_end - 1

        while page != end_page:
            text += doc[page].get_text()
            page += 1
        
        section_content_file = open(f'./textbooks/textbook/{chapter}_{chapter_name}/{sorted_index[i]}.txt', 'w', encoding='utf-8')
        section_content_file.write(text)

['1.1_Overview_of_Anatomy_and_Physiology', '1.2_Structural_Organization_of_the_Human_Body', '1.3_Functions_of_Human_Life', '1.4_Requirements_for_Human_Life', '1.5_Homeostasis', '1.6_Anatomical_Terminology', '1.7_Medical_Imaging']
['2.1_Elements_and_Atoms:_The_Building_Blocks_of_Matter', '2.2_Chemical_Bonds', '2.3_Chemical_Reactions', '2.4_Inorganic_Compounds_Essential_to_Human_Functioning', '2.5_Organic_Compounds_Essential_to_Human_Functioning']
['3.1_The_Cell_Membrane', '3.2_The_Cytoplasm_and_Cellular_Organelles', '3.3_The_Nucleus_and_DNA_Replication', '3.4_Protein_Synthesis', '3.5_Cell_Growth_and_Division', '3.6_Cellular_Differentiation']
['4.1_Types_of_Tissues', '4.2_Epithelial_Tissue', '4.3_Connective_Tissue_Supports_and_Protects', '4.4_Muscle_Tissue_and_Motion', '4.5_Nervous_Tissue_Mediates_Perception_and_Response', '4.6_Tissue_Injury_and_Aging']
['5.1_Layers_of_the_Skin', '5.2_Accessory_Structures_of_the_Skin', '5.3_Functions_of_the_Integumentary_System', '5.4_Diseases,_Disorders

dict_keys(['CHAPTER_1', 'CHAPTER_2', 'CHAPTER_3', 'CHAPTER_4', 'CHAPTER_5', 'CHAPTER_6', 'CHAPTER_7', 'CHAPTER_8', 'CHAPTER_9', 'CHAPTER_10', 'CHAPTER_11', 'CHAPTER_12', 'CHAPTER_13', 'CHAPTER_14', 'CHAPTER_15', 'CHAPTER_16', 'CHAPTER_17', 'CHAPTER_18', 'CHAPTER_19', 'CHAPTER_20', 'CHAPTER_21', 'CHAPTER_22', 'CHAPTER_23', 'CHAPTER_24', 'CHAPTER_25', 'CHAPTER_26', 'CHAPTER_27', 'CHAPTER_28'])